In [14]:
import pymorphy2

from spacy.lang import ru
from nltk.stem.snowball import SnowballStemmer 
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = set(nltk.corpus.stopwords.words('russian'))
Unused_chars=[':',',','.','-','\n','?','!']
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()


def tokenize(file_text):
    tokens = word_tokenize(file_text)
    tokens = [i for i in tokens if (i not in stop)]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    return tokens


def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [get_lemma(token) for token in tokens]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)




[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"\n        elif token.like_url:\n            lda_tokens.append('URL')\n        elif token.orth_.startswith('@'):\n            lda_tokens.append('SCREEN_NAME')\n"

In [25]:
#Testing zone, do not launch
import random
import glob
text_data = []
'''
for filename in glob.glob('Test\*.txt'):
    with open(filename,encoding='utf-8') as f:
        for line in f:
            tokens = prepare_text_for_lda(line)
            if random.random() > .001:
                print(tokens)
                text_data.append(tokens)
'''
with open('Test\Незнакомка.txt',encoding='utf-8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .001:
            print(tokens)
            text_data.append(tokens)



['вечер', 'ресторан']
['горячий', 'воздух', 'глухой']
['править', 'окрик', 'пьяный']
['весенний', 'тлетворный']
[]
['вдали', 'пыль', 'переулочный']
['скука', 'загородный']
['чуть', 'золотиться', 'крендель', 'булочный']
['раздаваться', 'детский', 'плач']
[]
['каждый', 'вечер', 'шлагбаум']
['заламывать', 'котелок']
['среди', 'канава', 'гулять', 'дама']
['испытанный', 'остряк']
[]
['озеро', 'скрипеть', 'уключина']
['раздаваться', 'женский', 'визг']
['небо', 'весь', 'приучить']
['бесмысленно', 'кривиться', 'диск']
[]
['каждый', 'вечер', 'друг', 'единственный']
['мыть', 'стакан', 'отражённый']
['влага', 'терпкий', 'таинственный']
['смирный', 'оглушить']
[]
['ряд', 'соседний', 'столик']
['лакей', 'сонный', 'торчать']
['пьяница', 'глаз', 'кролик']
['vino', 'veritas', 'кричать']
[]
['каждый', 'вечер', 'назначить']
['сниться']
['девичий', 'стан', 'шёлк', 'схватить']
['туманный', 'двигаться', 'окно']
[]
['медленно', 'пройти', 'пьяный']
['всегда', 'спутник', 'один']
['дышать', 'дух', 'туман']
['с

In [79]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')
import gensim
NUM_TOPICS = 1
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [80]:
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.018*"вечер" + 0.014*"пьяный" + 0.014*"каждый" + 0.011*"мой" + 0.011*"душа" + 0.011*"раздаваться" + 0.011*"берег" + 0.011*"окно" + 0.011*"терпкий" + 0.011*"очаровать"')


In [81]:
import re
symbols=['берег очарованный','незнакомка','диск','испытанные остряки','золотится крендель булочной','тлетворный']
symbol_counter = 0
counter=0
p=re.compile(r'"\w+"')
for topic in topics:
    #counter+=1
    lol = topic[1].replace(' +','')
    for topic_word in lol.split():
        counter+=1
        word = p.search(topic_word).group(0).replace('"','')
        print(word)
        for symbol in symbols:
            if word in symbol:
                symbol_counter+=1

'''
for topic in topics:
    counter+=1
    for topic_word in topic[1].split():
        print(topic_word)
        if topic_word in symbols:
            symbol_counter+=1
            continue
'''
print('Точность составляет '+str(symbol_counter/counter))

вечер
пьяный
каждый
мой
душа
раздаваться
берег
окно
терпкий
очаровать
Точность составляет 0.1


In [12]:
new_doc=''
with open('Test\Mi.txt',encoding='utf-8') as f:
    for line in f:
        new_doc+=line
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(0, 1), (1, 1), (2, 5), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 4), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 2), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1)]
[(0, 1.0)]


In [9]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
from urllib.error import URLError, HTTPError
import re

#def get_start_page(target):
    #return "http://www.symbolarium.ru/index.php/{0}".format(quote(target))
def get_start_page(target):
    #print("http://www.symbolarium.ru/index.php/{0}".format(quote(target)))
    return "http://www.symbolarium.ru/index.php/{0}".format(quote(target))
def get_page_url(target):
    print("http://www.symbolarium.ru{0}".format(quote(target)))
    return "http://www.symbolarium.ru{0}".format(quote(target))

html_page = urllib.request.urlopen(get_start_page("Символика"))
#html_page = urllib.request.urlopen("http://www.symbolarium.ru/index.php")
soup = BeautifulSoup(html_page)
#print(soup.prettify())
links = []
#for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
    #links.append(link.get('href'))
for link in soup.find_all('a', href=True):
    links.append(link.get('href'))
#for link in links:
    #print(link)
for link in links:
    try:
        SymbolFile = urllib.request.urlopen(get_page_url(link))        
    except URLError as e:
        #print('error')
        continue
    else:
        SymbolHtml = SymbolFile.read()
        SymbolFile.close()
        soup = BeautifulSoup(SymbolHtml,"lxml")
        text=soup.findAll("p")
        print(text)





http://www.symbolarium.ru/index.php/%D0%A1%D0%B8%D0%BC%D0%B2%D0%BE%D0%BB%D0%B8%D0%BA%D0%B0


C:\Users\nikov\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\nikov\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


http://www.symbolarium.ru%23mw-head
error
http://www.symbolarium.ru%23p-search
error
http://www.symbolarium.ru/index.php/%25D0%259A%25D0%25B0%25D1%2582%25D0%25B5%25D0%25B3%25D0%25BE%25D1%2580%25D0%25B8%25D1%258F%3A%25D0%25A1%25D0%25B8%25D0%25BC%25D0%25B2%25D0%25BE%25D0%25BB%25D0%25B8%25D0%25BA%25D0%25B0_%25E2%2580%2593_%25D0%259E%25D0%25B1%25D1%2589%25D0%25B8%25D0%25B5_%25D0%25BF%25D0%25BE%25D0%25BB%25D0%25BE%25D0%25B6%25D0%25B5%25D0%25BD%25D0%25B8%25D1%258F_%25D0%25B8_%25D1%2582%25D0%25B5%25D1%2580%25D0%25BC%25D0%25B8%25D0%25BD%25D0%25BE%25D0%25BB%25D0%25BE%25D0%25B3%25D0%25B8%25D1%258F
error
http://www.symbolarium.ru/index.php/%25D0%259A%25D0%25B0%25D1%2582%25D0%25B5%25D0%25B3%25D0%25BE%25D1%2580%25D0%25B8%25D1%258F%3A%25D0%25A1%25D0%25B8%25D0%25BC%25D0%25B2%25D0%25BE%25D0%25BB%25D0%25B8%25D0%25BA%25D0%25B0_%25E2%2580%2593_%25D0%25A3%25D0%25BD%25D0%25B8%25D0%25B2%25D0%25B5%25D1%2580%25D1%2581%25D0%25B0%25D0%25BB%25D1%258C%25D0%25BD%25D1%258B%25D0%25B5_%25D1%2581%25D0%25B8%25D0%25BC%2

error
http://www.symbolarium.ru/index.php%3Ftitle%3D%25D0%25A1%25D0%25B8%25D0%25BC%25D0%25B2%25D0%25BE%25D0%25BB%25D0%25B8%25D0%25BA%25D0%25B0%26action%3Dedit%26section%3D4
error
http://www.symbolarium.ru/index.php/%25D0%259A%25D0%25B0%25D1%2582%25D0%25B5%25D0%25B3%25D0%25BE%25D1%2580%25D0%25B8%25D1%258F%3A%25D0%259C%25D0%25B8%25D1%2580_%25D1%2587%25D0%25B5%25D0%25BB%25D0%25BE%25D0%25B2%25D0%25B5%25D0%25BA%25D0%25B0_%25E2%2580%2593_%25D0%2590%25D0%25BD%25D0%25B0%25D1%2582%25D0%25BE%25D0%25BC%25D0%25B8%25D1%258F_%25D0%25B8_%25D1%2581%25D0%25BE%25D1%2581%25D1%2582%25D0%25BE%25D1%258F%25D0%25BD%25D0%25B8%25D1%258F
error
http://www.symbolarium.ru/index.php/%25D0%259A%25D0%25B0%25D1%2582%25D0%25B5%25D0%25B3%25D0%25BE%25D1%2580%25D0%25B8%25D1%258F%3A%25D0%259C%25D0%25B8%25D1%2580_%25D1%2587%25D0%25B5%25D0%25BB%25D0%25BE%25D0%25B2%25D0%25B5%25D0%25BA%25D0%25B0_%25E2%2580%2593_%25D0%259E%25D0%25B4%25D0%25B5%25D1%258F%25D0%25BD%25D0%25B8%25D1%258F_%25D0%25B8_%25D1%2581%25D0%25B8%25D0%25BC%25D0%

error
http://www.symbolarium.ru/index.php/%25D0%259A%25D0%25B0%25D1%2582%25D0%25B5%25D0%25B3%25D0%25BE%25D1%2580%25D0%25B8%25D1%258F%3A%25D0%259F%25D0%25B5%25D1%2580%25D1%2581%25D0%25BE%25D0%25BD%25D0%25B0%25D0%25BB%25D0%25B8%25D0%25B9_%25E2%2580%2593_%25D0%259B%25D0%25B8%25D1%2587%25D0%25BD%25D0%25BE%25D1%2581%25D1%2582%25D0%25B8_%25D0%25B8%25D1%2581%25D1%2582%25D0%25BE%25D1%2580%25D0%25B8%25D1%2587%25D0%25B5%25D1%2581%25D0%25BA%25D0%25B8%25D0%25B5
error
http://www.symbolarium.ru/index.php/%25D0%259A%25D0%25B0%25D1%2582%25D0%25B5%25D0%25B3%25D0%25BE%25D1%2580%25D0%25B8%25D1%258F%3A%25D0%259F%25D0%25B5%25D1%2580%25D1%2581%25D0%25BE%25D0%25BD%25D0%25B0%25D0%25BB%25D0%25B8%25D0%25B9_%25E2%2580%2593_%25D0%259F%25D0%25B5%25D1%2580%25D1%2581%25D0%25BE%25D0%25BD%25D0%25B0%25D0%25B6%25D0%25B8_%25D0%25BB%25D0%25B8%25D1%2582%25D0%25B5%25D1%2580%25D0%25B0%25D1%2582%25D1%2583%25D1%2580%25D0%25BD%25D1%258B%25D0%25B5
error
http://www.symbolarium.ruhttp%3A//www.symbolarium.ru/index.php%3Ftitle%3D%D0

error
http://www.symbolarium.ru/index.php/%25D0%259A%25D0%25B0%25D1%2582%25D0%25B5%25D0%25B3%25D0%25BE%25D1%2580%25D0%25B8%25D1%258F%3A%25D0%2593%25D0%25B5%25D1%2580%25D0%25B0%25D0%25BB%25D1%258C%25D0%25B4%25D0%25B8%25D0%25BA%25D0%25B0_%25E2%2580%2593_%25D0%25A2%25D0%25B5%25D1%2580%25D0%25BC%25D0%25B8%25D0%25BD%25D0%25BE%25D0%25BB%25D0%25BE%25D0%25B3%25D0%25B8%25D1%258F_%25D0%25B8%25D0%25BD%25D0%25BE%25D1%258F%25D0%25B7%25D1%258B%25D1%2587%25D0%25BD%25D0%25B0%25D1%258F_/_%25D0%25BD%25D0%25B5%25D0%25BC%25D0%25B5%25D1%2586%25D0%25BA%25D0%25B0%25D1%258F
error
http://www.symbolarium.ru/index.php/%25D0%259A%25D0%25B0%25D1%2582%25D0%25B5%25D0%25B3%25D0%25BE%25D1%2580%25D0%25B8%25D1%258F%3A%25D0%2593%25D0%25B5%25D1%2580%25D0%25B0%25D0%25BB%25D1%258C%25D0%25B4%25D0%25B8%25D0%25BA%25D0%25B0_%25E2%2580%2593_%25D0%25A2%25D0%25B5%25D1%2580%25D0%25BC%25D0%25B8%25D0%25BD%25D0%25BE%25D0%25BB%25D0%25BE%25D0%25B3%25D0%25B8%25D1%258F_%25D0%25B8%25D0%25BD%25D0%25BE%25D1%258F%25D0%25B7%25D1%258B%25D1%2587